# FINN - End-to-End Flow
-----------------------------------------------------------------
This notebook gives an overview about the end to end flow of FINN. From loading an ONNX model from Brevitas, followed by the numerous transformations in FINN and up to the generation of a bitstream that can be used to load an FPGA. 

## Outline
-------------
* Preparation of model to pass it to FINN
* FINN transformations
    * first transformations
    * Streamline
    * last transformations
* Verification
* Bitstream generation
